## The network flow problem. Assume one kind of supply and one drone per warehouse.

In [121]:
# this solution uses NamedArrays, which are arrays indexed over sets for both x and y dimensions.

using JuMP, Clp, NamedArrays

warehouses = [ 1,  2,  3]
hospitals = [:A, :B, :C, :D, :E]


cost_per_flight_permile = 4 

dist = NamedArray( [8 15 50 15 76 ; 10 17 20 67 34; 30 26 15 24 56], (warehouses,hospitals), ("Warehouses","Hospitals") )
supply = Dict(zip( warehouses, [11 45 33] ))#unlimited supply

demand = Dict(zip( hospitals,  [4 5 7 5 3] ))

default_cost = [100 143 134 456 312]

m = Model(solver=ClpSolver())

@variable(m, x[warehouses,hospitals] >= 0)

@constraint(m, sup[i in warehouses], sum(x[i,j] for j in hospitals) <= supply[i] )   # supply constraint
@constraint(m, dem[j in hospitals], sum(x[i,j] for i in warehouses) == demand[j] )   # demand constraint


@objective(m, Min, cost_per_flight*sum( x[i,j]*dist[i,j] for i in warehouses, j in hospitals ) )      # minimize transportation cost

status = solve(m)

println(status)

# nicely formatted solution
solution = NamedArray( Int[getvalue(x[i,j]) for i in warehouses, j in hospitals], (warehouses,hospitals), ("warehouses","hospitals") )
println( solution )
println()
println("Total cost will be \$", getobjectivevalue(m))

Optimal
3×5 Named Array{Int64,2}
warehouses ╲ hospitals │ :A  :B  :C  :D  :E
───────────────────────┼───────────────────
1                      │  4   2   0   5   0
2                      │  0   3   0   0   3
3                      │  0   0   7   0   0

Total cost will be $1580.0


## Complication: first part. We are going to use this first part to determine how many warehouses to use and find out the optimal allocations for supply delivery



In [133]:
# try to complicate: activation costs of warehouses; complicate the types of supplies; add the cost because of default

# drone number, capacity, type, range&charging station;
# time variable: stage
# supply types

using JuMP, Gurobi, NamedArrays

#sources and destinations
warehouses = [ 1,  2,  3, 4, 5, 6]
hospitals = [:A, :B, :C, :D, :E, :F, :G]

#costs
warehouse_costs = [900 1300 1600 1350 1200 1500]
cost_per_flight = 2
default_cost = [460 620 300 500 1400 450 620]
dist = NamedArray( [37 57 23 26 27 19 34;
        32 34 77 147 43 67 34; 
        45 65 65 26 145 24 56; 
        123 45 32 45 16 34 15; 
        52 63 12 56 45 133 54;
        65 76 24 34 53 124 54;
        ], (warehouses,hospitals), ("Warehouses","Hospitals") )

supply = Dict(zip( warehouses, [21 27 33 31 14 32] ))#unlimited supply
demand = Dict(zip( hospitals,  [14 15 7 35 3 14 26] ))

#Network-flow model
m = Model(solver=GurobiSolver())

@variable(m, x[warehouses,hospitals] >= 0)
@variable(m, activation[1:6] >= 0, Bin)   #if activation[i] = 1, means the warehouse is activated

@constraint(m, sup[i in warehouses], sum(x[i,j] for j in hospitals) <= supply[i] )   # supply constraint
@constraint(m, dem[j in hospitals], sum(x[i,j] for i in warehouses) == demand[j] )   # demand constraint

for i in 1:6
    @constraint(m, sum(x[i,j] for j in hospitals) <= 99999*activation[i] ) #if used, then activated
end

@objective(m, Min, sum(warehouse_costs[i]*activation[i] for i in warehouses)
    + cost_per_flight*sum(x[i,j]*dist[i,j] for i in warehouses, j in hospitals ) )      # minimize transportation cost

status = solve(m)

println(status)

# nicely formatted solution
solution = NamedArray([getvalue(x[i,j]) for i in warehouses, j in hospitals], (warehouses,hospitals), ("warehouses","hospitals") )
println( solution )
println()
println("Total cost will be \$", getobjectivevalue(m))

Academic license - for non-commercial use only
Optimize a model with 19 rows, 48 columns and 132 nonzeros
Variable types: 42 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 13 rows, 48 columns, 90 nonzeros
Variable types: 42 continuous, 6 integer (6 binary)

Root relaxation: objective 1.074440e+04, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 10744.4032    0    2          - 10744.4032      -     -    0s
H    0     0                    12144.000000 10744.4032  11.5%     -    0s
     0     0 10782.2500    0    1 12144.0000 10782.2500  11.2%     -    0s
H    0     0                    11682.000000 10782.2500  7.70%     -    0s
     0

## Complication: second part. In this part, we are going to decide the types of drone that are going to be used in the assignment.

In [122]:
using JuMP, Gurobi
#Ipopt

## unlimited constraints:
#droneNum = ?
#droneRange = ?
#MaxNumDrones = ?
#MaxDroneCapacities(this is going to be a vector. Each element represents the max capacity of a kind of drone)
#TimeCost = ? this is the cost for "pilto" for drones, and since the hospital needs daily deman, I think this can be a time
#constraint for the daily demand
#...

#daily demands for each hospital: each row represents a hospital, each colum represents demand for supply A, B, C respectively
demands = [4 5 7;
            2 4 5;
            3 5 2;
            1 5 6;
            3 5 2]

#madeup package weight: represents supplies A, B, C respectively in kilograms(or pounds, whatever)
med_package_weight = [2 2 3]

#capacity for drone (suppose only one type of the drone)
capacity = 10

m1 = Model(solver = GurobiSolver())
@variable(m1, DroneNum >= 0, Int)
@variable(m1, supplyPerDrone[1:3] >=0 , Int)
for j in 1:5
    for i in 1:3
        @constraint(m1, DroneNum*supplyPerDrone[i] >= demands[j, i])
    end
end
@constraint(m1, sum(supplyPerDrone[i]*med_package_weight[i] for i in 1 : 3) <= capacity)

@objective(m1, Min, DroneNum)

solve(m1)

println("Lest possible number of drone is: ",getobjectivevalue(m1))

Academic license - for non-commercial use only
Optimize a model with 1 rows, 4 columns and 3 nonzeros
Model has 15 quadratic constraints
Variable types: 0 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
  QRHS range       [1e+00, 7e+00]
Presolve time: 0.01s
Presolved: 1 rows, 4 columns, 3 nonzeros
Variable types: 0 continuous, 4 integer (0 binary)

Root relaxation: objective 0.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.48718    0    4          -    2.48718      -     -    0s
H    0     0                       5.0000000    2.48718  50.3%     -    0s
     0     0    3.82353    0    4    5.00000    3.82353  23.5%     -    0s
     0     0     cutoff  